In [13]:
import matplotlib
from pylab import *
import networkx as nx
import random as rd
import pandas as pd
import numpy as np

In [3]:
matplotlib.use('TkAgg')

n = 200
p_e = 0.05
p_i = 0.04
p_r = 0.5

def initialize():
    global g
    g = nx.karate_club_graph()
    g.pos = nx.spring_layout(g)
    for i in g.nodes:
        g.nodes[i]['state'] = 1 if random() < p_e else 0

def observe():
    global g
    cla()
    nx.draw(g, vmin = 0, vmax = 1,
        node_color = [g.nodes[i]['state'] for i in g.nodes],
        pos = g.pos)

def update():
    # Modified from asynchronous updating to synchronous updating
    global g
    all_nodes = list(g.nodes)
    
    for node in all_nodes:
        if g.nodes[node]['state'] == 0:
            for neighbor in list(g.neighbors(node)):
                if g.nodes[neighbor]['state'] == 1:
                    g.nodes[node]['state'] = 1 if random() < p_i else 0
        else:
            g.nodes[node]['state'] = 0 if random() < p_r else 1

import pycxsimulator
pycxsimulator.GUI().start(func=[initialize, observe, update])

/Users/rebeccamqamelo/opt/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


### Question: Why does using synchronous or asynchronous updating make a difference?
In asynchronous updates, nodes to NOT get updated at the same time, therefore the assumption that "under that same conditions similar nodes will react similarly" doesn't hold. At any time step, the update state of nodes depends on the order of updates between nodes.

### Question: For the mean field approximation described in Section 18.5, why is it appropriate to use the synchronous update method and not the asynchronous one?
Synchronous updating satisfies the above assumption more strongly

In [8]:
er = nx.erdos_renyi_graph(1000, 0.04)
ws = nx.watts_strogatz_graph(1000,40,0.1)
ba = nx.barabasi_albert_graph(1000,20)

In [9]:
# Average degree of a graph
def exp_k(g):
    n = len(g.nodes())
    e = len(list(g.edges))
    print ("Total edges: ", e)
    k = e * 2/ n
    print ("Average degree: ", k)
    return k

In [10]:
# Average degree of each neighbor in a graph
def exp_k_prime(g):
    node = 0
    deg = 0
    for edge in g.edges():
        for neighbor in edge:
            deg += len(g.edges(neighbor))
            node += 1
    k_p = deg/node
    print ("Average neighbor degree: ", k_p)
    return k_p

### Question: How does the average degree of neighbors (the number of friends of your friends) compare to the average degree of the graph (your number of friends)?

The table below shows that for ER and WS, the ave_degree of the graph is very similar to the ave_degree of the entire graph. However, for BA, there is a significant difference between the two due to prefernetial attachment – some nodes have a very high degree while far more have a very low degree.

In [21]:
graphs = [er, ws, ba]
graph_type = ['erdos_renyi', 'watts_strogatz', 'barabasi_albert']
ave_degree = [exp_k(graph) for graph in graphs]
ave_degree_neighbor = [exp_k_prime(graph) for graph in graphs]
ratio = [exp_k(graph)/exp_k_prime(graph) for graph in graphs]

df = pd.DataFrame(np.array([ave_degree, ave_degree_neighbor, ratio]), 
                  columns=graph_type, index=['ave_deg', 'ave_neigh_deg', 'ratio'])
df

Total edges:  20065
Average degree:  40.13
Total edges:  20000
Average degree:  40.0
Total edges:  19600
Average degree:  39.2
Average neighbor degree:  41.06299526538749
Average neighbor degree:  40.09245
Average neighbor degree:  62.01112244897959
Total edges:  20065
Average degree:  40.13
Average neighbor degree:  41.06299526538749
Total edges:  20000
Average degree:  40.0
Average neighbor degree:  40.09245
Total edges:  19600
Average degree:  39.2
Average neighbor degree:  62.01112244897959


,erdos_renyi,watts_strogatz,barabasi_albert
ave_deg,40.130000,40.000000,39.200000
ave_neigh_deg,41.062995,40.092450,62.011122
ratio,0.977279,0.997694,0.632145
